In [ ]:
# import pandas as pd
# import numpy as np
# import warnings
# warnings.filterwarnings('ignore')

# import pandas as pd
# import warnings

# warnings.filterwarnings('ignore')


# your_file_name = 'Point_Visuals_KaylanBigun_NathanTrouve.csv'
# data = pd.read_csv(your_file_name)


# def gen_serve_dist_bar(path, player):
#     # Load the data
#     events = pd.read_csv(path)
#     events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    
#     # Filter the events to get only serves for player1
#     filtered_events = events[
#         (events['shotInRally'] == 1) &
#         (events['serverName'] == player) &
#         ((events['firstServeIn'] == 1.0) | (events['secondServeIn'] == 1.0))
#     ]
    
#     valid_placements = ['Wide', 'T', 'Body']
#     filtered_events = filtered_events[filtered_events['serveInPlacement'].isin(valid_placements)]
#     filtered_events['Zone'] = filtered_events['side'] + " " + filtered_events['serveInPlacement']

#     serve_counts = filtered_events.groupby(['side', 'Zone']).size()
#     side_totals = serve_counts.groupby(level=0).sum()

#     zone_percentages = (serve_counts / side_totals) * 100
#     zone_percentages = zone_percentages.round().astype(int)
#     zone_percentages = zone_percentages.reset_index()

#     serve_dist = pd.DataFrame({
#         "Zone": zone_percentages['Zone'],
#         "Percentage": zone_percentages[0].astype(str) + '%'
#     })

#     serve_dist['Server'] = [player, "", "", "", "", ""]
    
#     serve_dist_bar_json = serve_dist.to_json(orient='records')
#     with open('serve_dist_bar.json', 'w') as f:
#         f.write(serve_dist_bar_json)

# player = "Kaylan Bigun"
# path = "Shot_Visuals_KaylanBigun_NathanTrouve.csv"
# gen_serve_dist_bar(path, player)

In [ ]:
# def gen_serve_dist_bar(path, player):
#         data = pd.read_csv(path)


# server_data = data.query('serverName == "Kaylan Bigun"').copy()

# first_server_data = server_data[server_data['serveResult'] == '1st Serve In']
# first_serve_table = (first_server_data['player1ServePlacement'].value_counts(normalize=False)*1).round()

# second_server_data = server_data[server_data['serveResult'] == '2nd Serve In']
# second_serve_table = (second_server_data['player1ServePlacement'].value_counts(normalize=False)*1).round()

# player = "Kaylan Bigun"

# # Function to create JSON-like structure for serves
# def create_json(serve_table, serve_type, server_name, total_serves):
#     json_output = []
#     for i, (zone, count) in enumerate(serve_table.items()):
#         percentage = (count / total_serves) * 100
#         entry = {
#             "Serve": serve_type,
#             "Zone": zone.replace(":", ""),
#             "Percentage": f"{int(round(percentage))}%",
#             "Server": server_name if i == 0 else ""  # Server name only for the first entry
#         }
#         json_output.append(entry)
#     return json_output

# # Calculate total serves for percentage calculations
# total_first_serves = first_serve_table.sum()
# total_second_serves = second_serve_table.sum()

# # Create JSON for both first and second serves
# serve_dist = create_json(first_serve_table, "First", player, total_first_serves) + \
#              create_json(second_serve_table, "Second", player, total_second_serves)

# # Convert to JSON format and save to file
# serve_dist_bar_json = json.dumps(serve_dist, indent=4)

# with open('serve_dist_bar.json', 'w') as f:
#     f.write(serve_dist_bar_json)

# print("JSON file saved as 'serve_dist_bar.json'")


# # player = "Kaylan Bigun"
# # path = "Shot_Visuals_KaylanBigun_NathanTrouve.csv"
# # gen_serve_dist_bar(path, player)

JSON file saved as 'serve_dist_bar.json'


In [4]:
# import pandas as pd
# import json
# import warnings

# # List of CSV file names
# csv_files = ["Point_Visuals_KaylanBigun_MaxiHomberg.csv"
#                         ,"Point_Visuals_KaylanBigun_MiguelPerezPena.csv"
#                         ,"Point_Visuals_KaylanBigun_NathanTrouve.csv"
#                         ,"Point_Visuals_KaylanBigun_SavriyanDanilov.csv"]

# # Read and concatenate the CSV files
# df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# # Filter rows where pointScore is "30-30" or "40-30"
# filtered_df = df[df["pointScore"].isin(["30-30", "40-30", "30-40", "40-40"])]

In [5]:
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
'''
Code to get 1st / 2nd serve data from point visuals in a JSON format to create d3.js visual
'''
def gen_serve_dist_bar(path, player):
    # Read data
    data = pd.read_csv(path)
    data = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    filtered_df = data[data["pointScore"].isin(["30-30", "40-30", "30-40", "40-40"])]  

    
    # Filter data for the given player
    server_data = filtered_df.query('serverName == @player').copy()
    
    # Separate first and second serves
    first_server_data = server_data[server_data['serveResult'] == '1st Serve In']
    second_server_data = server_data[server_data['serveResult'] == '2nd Serve In']
    
    # Count serve placements
    first_serve_table = first_server_data['player1ServePlacement'].value_counts(normalize=False)
    second_serve_table = second_server_data['player1ServePlacement'].value_counts(normalize=False)
    
    # Calculate totals for Ad and Deuce sides separately for first serves
    first_ad_total = first_serve_table[first_serve_table.index.str.contains("Ad")].sum()
    first_deuce_total = first_serve_table[first_serve_table.index.str.contains("Deuce")].sum()
    
    # Calculate totals for Ad and Deuce sides separately for second serves
    second_ad_total = second_serve_table[second_serve_table.index.str.contains("Ad")].sum()
    second_deuce_total = second_serve_table[second_serve_table.index.str.contains("Deuce")].sum()
    
    # Function to create JSON-like structure for serves
    def create_json(serve_table, serve_type, server_name, ad_total, deuce_total):
        json_output = []
        for i, (zone, count) in enumerate(serve_table.items()):
            if "Ad" in zone:
                percentage = (count / ad_total) * 100 if ad_total > 0 else 0
            elif "Deuce" in zone:
                percentage = (count / deuce_total) * 100 if deuce_total > 0 else 0
            else:
                percentage = 0
            
            entry = {
                "Serve": serve_type,
                "Zone": zone.replace(":", ""),
                "Percentage": f"{int(round(percentage))}%",
                "Server": server_name if i == 0 else "",  # Server name only for the first entry
                "Count": count  # Include the count of serves for each zone
            }
            json_output.append(entry)
        return json_output
    
    # Make sure that empty rows are handled properly + in correct order for d3.js
    # Above code doesn't get distributions(ex: Ad: Wide) with values 0, messes up d3.js code. Ensure we add these(even with count 0) to the json.
    types = ['Ad: Wide','Ad: Body','Ad: T','Deuce: Wide','Deuce: Body','Deuce: T']
    for t in types:
        if (first_serve_table.get(t,0) == 0):
            first_serve_table[t] = 0
        if (second_serve_table.get(t,0) == 0):
            second_serve_table[t] = 0
    
    # Correct Order for d3.js
    first_serve_table = first_serve_table.reindex(types)
    second_serve_table = second_serve_table.reindex(types)
    
    
    # Create JSON for both first and second serves
    serve_dist = create_json(first_serve_table, "First", player, first_ad_total, first_deuce_total) + \
                 create_json(second_serve_table, "Second", player, second_ad_total, second_deuce_total)

    # Convert to JSON format and save to file
    serve_dist_bar_json = json.dumps(serve_dist, indent=4)
    fout = 'serve_dist_bar' + '.json'
    with open(fout, 'w') as f:
        f.write(serve_dist_bar_json)
    
    print("JSON file saved as {fout}")

# Usage

# To run multiple teams at once. zip([l1],[l2]) -> (path, team)
    path = ["Point_Visuals_KaylanBigun_MaxiHomberg.csv"]
    player = "Kaylan Bigun"
    gen_serve_dist_bar(path, player)
